In [1]:
%%capture
!git lfs install
!git clone https://github.com/margavsavsani/HateXplain
!pip install ekphrasis
!pip install transformers
!pip install GPUtil
!pip install sentencepiece

In [2]:
cd HateXplain

/content/HateXplain


In [3]:
%%capture
from Models.bertModels import *
from Models.utils import *
from Preprocess.attentionCal import *
from Preprocess.dataCollect import *
from Preprocess.preProcess import *
from Preprocess.spanMatcher import *
from TensorDataset.dataLoader import *
from TensorDataset.datsetSplitter import *
from manual_training_inference import *
from predict_single import *

In [4]:
params_data={
    'include_special':False, 
    # 'bert_tokens':False,
    'bert_tokens':True,
    'type_attention':'softmax',
    'set_decay':0.1,
    'majority':2,
    'max_length':128,
    'variance':5,
    'window':4,
    'alpha':0.5,
    'p_value':0.8,
    'method':'additive',
    'decay':False,
    'normalized':False,
    'not_recollect':True,
}

common_hp={
    'is_model':True,
    'logging':'local',  ###neptune /local
    'learning_rate':2e-5,  ### learning rate 2e-5 for bert 0.001 for gru
    'epsilon':1e-8,
    'batch_size':16,
    'to_save':True,
    'epochs':2,
    'auto_weights':True,
    'weights':[1.0795518,0.82139814,1.1678787 ],
    'model_name':'birnn',
    'random_seed':42,
    'num_classes':3,
    'att_lambda':0.001,
    # 'device':'cuda',
    'device':'cud',
    'train_att':True

}
    
    
params_bert={
    'path_files':'bert-base-uncased',
    'what_bert':'weighted',
    'save_only_bert':False,
    'supervised_layer_pos':11,
    'num_supervised_heads':6,
    'dropout_bert':0.1
 }


params_other = {
        "vocab_size": 0,
        "padding_idx": 0,
        "hidden_size":64,
        "embed_size":0,
        "embeddings":None,
        "drop_fc":0.2,
        "drop_embed":0.2,
        "drop_hidden":0.1,
        "train_embed":False,
        "seq_model":"gru",
        "attention":"softmax"
}


if(params_data['bert_tokens']):
    for key in params_other:
        params_other[key]='N/A'
else:
    for key in params_bert:
        params_bert[key]='N/A'


def Merge(dict1, dict2,dict3, dict4): 
    res = {**dict1, **dict2,**dict3, **dict4} 
    return res 

params = Merge(params_data,common_hp,params_bert,params_other)


dict_data_folder={
      '2':{'data_file':'Data/dataset.json','class_label':'Data/classes_two.npy'},
      '3':{'data_file':'Data/dataset.json','class_label':'Data/classes.npy'}
}

In [6]:
params['variance']=5
params['epochs']=2
params['to_save']=False
params['num_classes']=3
params['data_file']=dict_data_folder[str(params['num_classes'])]['data_file']
params['class_names']=dict_data_folder[str(params['num_classes'])]['class_label']
if(params['num_classes']==2 and (params['auto_weights']==False)):
      params['weights']=[1.0,1.0]
        
#for att_lambda in [0.001,0.01,0.1,1,10,100]
params['att_lambda']=0.001

params['device']='cuda'

In [7]:
torch.autograd.set_detect_anomaly(True)
if torch.cuda.is_available() and params['device']=='cuda':    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    ##### You can set the device manually if you have only one gpu
    ##### comment this line if you don't want to manually set the gpu
    deviceID = get_gpu()
    torch.cuda.set_device(deviceID[0])
    ##### comment this line if you don't want to manually set the gpu
    #### parameter required is the gpu id
    #torch.cuda.set_device(0)
    
else:
    print('Since you dont want to use GPU, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Found a gpu
We will use the GPU: 0 Tesla T4


In [8]:
# train_model(params,device)

In [ ]:
# from prettytable import PrettyTable

# def count_parameters(model):
#     table = PrettyTable(["Modules", "Parameters"])
#     total_params = 0
#     for name, parameter in model.named_parameters():
#         if not parameter.requires_grad: continue
#         params = parameter.numel()
#         table.add_row([name, params])
#         total_params+=params
#     print(table)
#     print(f"Total Trainable Params: {total_params}")
#     return total_params

In [ ]:
# params["variance"]=4

In [12]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

True

In [103]:
import numpy as np
import pandas as pd
from IPython.display import HTML as html_print
def cstr(s, color='white'):
  return "<text style=color:{}>{}</text>".format(color, s)
def display_results(classes, norm_attn):
  idx = np.argpartition(norm_attn, -4)[-4:]
  our_rationales = idx[np.argsort(-norm_attn[idx])]

  print(f"      Label : {classes}\n")
  display(pd.DataFrame([(sentence[i], norm_attn[i]) for i in our_rationales], columns=['rationale', 'attention']))
  print('\n')

  marked_sentence = []
  for i in range(len(sentence)):
    word = sentence[i]
    if i in our_rationales:
      marked_sentence.append(cstr(word, color='lime'))
    else:
      marked_sentence.append(word)
  return html_print(cstr(' '.join(marked_sentence)))

In [118]:
# sample toxic entries to demo on, picked from the hate-xplain dataset itself:
sample = [
    'this bitch in whataburger eating a burger with no top bun and holding it palm up i hate white bitches',
    'tired i can not support abortion from a moral standpoint wired i can support it morally easy stop looking as those little niglets as future doctors and see them for what they really are future rapists of white women and future welfare recipient of white tax dollars and future democrat voters',
    'all of my exes were cute but they were whores i guess i only attract fly looking thots',
]

In [125]:
%%capture
# sentence = word_tokenize(sample[0])
sentence = word_tokenize("the jews are again using holohoax as an excuse  to spread their agenda hitler should have eradicated them")
_, classes, attn_values = pred_model(params,device,sentence)
norm_attn = np.array([(attn)/sum(attn_values) for attn in attn_values])

In [126]:
display_results(classes, norm_attn)

      Label : offensive



,rationale,attention
0,holohoax,0.206409
1,jews,0.159623
2,are,0.093108
3,hitler,0.086157
